# before we start drimm-synteny anaslysis, we need to generate synOG table using genespace_parser()
    refer to genespace_parser.ipynb
    be carefull about how speciese name and gene name are connected ("@" or "_")
1. use “processOrthofinder_tao.py” to process the genespace pangenome output to get “drimm.sequence” —> be careful about the copy_ratio

2. use “drimm-synteny” on windows to built up drimm blocks. —→ this current only works on windows, we can use it on unix with Wine software, but on Merian, I have issues with X11 library which can not find by Wine

3. use “processDrimm_tao.py” on the outputs from last step to obtain final blocks for IAGS

4. use IAGS to construct ancestral genomes; I also tried mgra (by index and reformat the blocks) but it doesn’t work.
    
from here we can use either iags <option -ff> or grimm to calculate genome shuffling event’s:
    
5. ***a***) use “standardize_drimmBlocks.py” to index and reformat the drimm genomes including ancestors and also extent genomes; then use grimm to characterise genome rearrangements; ***b***) use iags to calculate directly

6. plotting and interpretation using iags (option -p); before this we need use “processGenenumber_tao.py” to generate the data needed by iags

In [2]:
# get dependencies ready
import os
from pathlib import Path
from utils import processLCSAndFirstFilter_2sp as plff
from utils import processFinalFilter_2sp as pff
import shutil
from processDrimm_tao_2sp import readSequence,prepare_dir,write_raw_blocks,processDrimm
from pathlib import Path
import pandas as pd
import numpy as np
from modules_paper import prepare_OG,prepare_meta,make_comBed,combed_parser,grimmBlock_parser,getFilterSequence,getAllSequence,outSequence,standarlize_drimmblocks,get_block_info,get_chromosome_info
from parse_genespaceBED_for_drimmBED_paper import make_index,make_drimmBED,make_genome_meta,make_chromosome_meta,get_chromosome_lenght,parse_genespaceBED_for_drimmBED

In [2]:
# get input data ready
sp='sp46'
index_file = "/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/Asteraceae_phylogenomics/pipeline/Smar.EM05.v1.genome.fa.fai"

workdir = "/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/Asteraceae_phylogenomics/pipeline/workdir"
BED_folder = "/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/Asteraceae_phylogenomics/pipeline/"
if not workdir[-1] == "/": workdir += "/"
if not BED_folder[-1] == "/": BED_folder += "/"

In [3]:
# step0, first let's prepare bed and meta data
## make index file
df_index = make_index(sp,index_file) #your genome index and genome name you want to use

## make bed, genome_meta, chromosome_meta
DIC = {f"{row['sp']}@{row['chr']}": row['length'] for _, row in df_index.iterrows()}
bed_dir = workdir + "bed/"
if not os.path.exists(bed_dir):
    os.makedirs(bed_dir)
    make_drimmBED(BED_folder, DIC, bed_dir)
else:
    print(bed_dir + " exists, let's generate bed file in this floder")
    make_drimmBED(BED_folder, DIC, bed_dir)

'''
for file in os.listdir(BED_folder):
    if file.endswith(".bed"):
        genome_name = os.path.splitext(file)[0]
        bed = BED_folder + file
        new_bed = new_bed_path + genome_name + ".bed"
        if not os.path.exists(new_bed):
            print(new_bed + " is new, adding to database ...")
            chr_info,df = parse_genespaceBED_for_drimmBED(bed,DIC)
            df.to_csv(new_bed, header=None, index=None, sep='\t')
            chr_size = ["_".join(c.split("_")[0:-1]) for c in chr_info] # remove the chr length (e.g., "sp03v1@a11_1_1679_69615488")
            list_ = [genome_name] + chr_size
            chromosome_meta_dict.append(list_)
            #f.write(genome_name + "\t" + "\t".join(chr_size) + "\n")
        else:
            print(new_bed + " exists, let's just make chromosome_meta!")
            chr_info,df = parse_genespaceBED_for_drimmBED(bed,DIC)
            chr_size = ["_".join(c.split("_")[0:-1]) for c in chr_info] # remove the chr length (e.g., "sp03v1@a11_1_1679_69615488")
            list_ = [genome_name] + chr_size
            chromosome_meta_dict.append(list_)
'''

/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/Asteraceae_phylogenomics/pipeline/workdir/bed/ exists, let's generate bed file in this floder
/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/Asteraceae_phylogenomics/pipeline/workdir/bed/sp03v5.bed exists!
/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/Asteraceae_phylogenomics/pipeline/workdir/bed/sp46.bed exists!


'\nfor file in os.listdir(BED_folder):\n    if file.endswith(".bed"):\n        genome_name = os.path.splitext(file)[0]\n        bed = BED_folder + file\n        new_bed = new_bed_path + genome_name + ".bed"\n        if not os.path.exists(new_bed):\n            print(new_bed + " is new, adding to database ...")\n            chr_info,df = parse_genespaceBED_for_drimmBED(bed,DIC)\n            df.to_csv(new_bed, header=None, index=None, sep=\'\t\')\n            chr_size = ["_".join(c.split("_")[0:-1]) for c in chr_info] # remove the chr length (e.g., "sp03v1@a11_1_1679_69615488")\n            list_ = [genome_name] + chr_size\n            chromosome_meta_dict.append(list_)\n            #f.write(genome_name + "\t" + "\t".join(chr_size) + "\n")\n        else:\n            print(new_bed + " exists, let\'s just make chromosome_meta!")\n            chr_info,df = parse_genespaceBED_for_drimmBED(bed,DIC)\n            chr_size = ["_".join(c.split("_")[0:-1]) for c in chr_info] # remove the chr leng

In [3]:
# step1 process synOG
## get dependent info ready, read in synOG table
from processOrthofinder_tao import get_group_dir,get_final_group,main

## read in synOG table and adjust name
orthogroups = "/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/2_analysis/1_genespace/run30/pangenome_sp03v5_synOG_2sp_syn.txt"
## be careful with this orthogroups table; in terms of the name "sp03v5" vs ""sp03_v5"", and "sp03v5@gene1" vs "sp03v5_gene1"; we need to make this consistent in genespace_parser()
ortho = pd.read_csv(orthogroups, sep='\t', dtype = str, keep_default_na=False)
ortho2 = ortho[ortho["interpChr"] != ""]

ortho2.rename(columns={'sp03_v5': 'sp03v5', 'sp03_v1': 'sp03v1'}, inplace=True)
ortho2['sp03v5'] = ortho2['sp03v5'].str.replace('sp03_v5@', 'sp03v5@')
ortho2.head()

,pgID,interpChr,repGene,sp46,sp03v5
0,1,a1,Alap.ptg000007l.g159.t1,sp46@Smar01g000670.1,sp03v5@Alap.ptg000007l.g159.t1
1,2,a1,Alap.ptg000007l.g160.t1,sp46@Smar01g000680.1,sp03v5@Alap.ptg000007l.g160.t1
2,4,a1,Alap.ptg000007l.g161.t1,sp46@Smar01g000690.1,sp03v5@Alap.ptg000007l.g161.t1
3,9,a1,Alap.ptg000007l.g162.t1,sp46@Smar01g000700.1,sp03v5@Alap.ptg000007l.g162.t1
4,10,a1,Alap.ptg000007l.g163.t1,sp46@Smar01g000710.1,sp03v5@Alap.ptg000007l.g163.t1


In [4]:
## process synOG table, call pairwise orthologs for given ref-target pair
##target = ["sp28","sp24","sp43","sp44","sp18","sp34","sp14","sp10","sp08","sp35","sp05a","sp39","sp37","sp11","sp13","sp22","sp21","sp29","sp30","sp03_v5","sp03_v1","sp03","sp41","sp07","sp15"]
target = ["sp46"]
reference = "sp03v5"
OG_type = "synOG" # always check the varable "sp_ratio"

genome_meta = make_genome_meta("sp46",1,17)
chromosome_meta = make_chromosome_meta(BED_folder,DIC)
print(chromosome_meta)

outdir = workdir + "out1"
if outdir[-1] != "/": outdir += "/"
if bed_dir[-1] != "/": bed_dir += "/"

for species in target:
    if not species == reference: 
        species_list = [reference, species]
        subdir = outdir + species + "/1_SynOG/"
        if not Path(subdir).exists():
            os.makedirs(subdir)
            sp,sp_ratio,sp_chr_number,long_chr_list,gff_list = prepare_meta(species_list,genome_meta,chromosome_meta,OG_type,bed_dir)
            #print(sp)
            #print(sp_chr_number)
            print(gff_list,long_chr_list)
            ortho3 = prepare_OG(ortho2,sp,OG_type)
            group_dir = get_group_dir(ortho3,sp)
            finalGroup = get_final_group(group_dir,sp_ratio)
            main(bed_dir,subdir,group_dir,finalGroup,gff_list,long_chr_list)
        else: print(species + " seems already done, please check!")

[['sp03v5', 'sp03v5@a11_1_1679', 'sp03v5@a5_2_1622', 'sp03v5@a12_3_1595', 'sp03v5@b11_4_1586', 'sp03v5@c12_5_1441', 'sp03v5@c11_6_1341', 'sp03v5@c13_7_1301', 'sp03v5@b13_8_1246', 'sp03v5@a13_9_1243', 'sp03v5@a10_10_1189', 'sp03v5@b12_11_1137', 'sp03v5@c14_12_1116', 'sp03v5@a4_13_1044', 'sp03v5@b6_14_1044', 'sp03v5@a14_15_1031', 'sp03v5@c5_16_1030', 'sp03v5@c4_17_964', 'sp03v5@a2_18_931', 'sp03v5@b5_19_931', 'sp03v5@a7_20_917', 'sp03v5@b10_21_917', 'sp03v5@b4_22_903', 'sp03v5@c6_23_874', 'sp03v5@a1_24_861', 'sp03v5@b7_25_836', 'sp03v5@b2_26_822', 'sp03v5@c2_27_786', 'sp03v5@c7_28_743', 'sp03v5@c10_29_728', 'sp03v5@c9_30_689', 'sp03v5@a3_31_679', 'sp03v5@b1_32_677', 'sp03v5@c8_33_664', 'sp03v5@a6_34_634', 'sp03v5@b14_35_614', 'sp03v5@a9_36_577', 'sp03v5@b8_37_577', 'sp03v5@c15_38_565', 'sp03v5@c1_39_515', 'sp03v5@b3_40_475', 'sp03v5@c3_41_464', 'sp03v5@a8_42_454', 'sp03v5@b9_43_410', 'sp03v5@a15_44_266', 'sp03v5@b15_45_110'], ['sp46', 'sp46@chr01_1_4688', 'sp46@chr05_2_4559', 'sp46@chr02

### Step2 perform drimm-synteny in windows
1. transfer the data from last step to windows
   #put the whole directory (e.g., genespace26) in C:\Users\fengtao\Desktop\drimm_batch\
2. double click the windows bat file "drimm-synteny"; the tool will loop the genomes in "genespace26" and perform calculation
   
   **input poped options:**
     + The path of drimm.sequence <drimm.sequence>
     + The output directory <.>
     + cycleLengthThreshold controls the continuity of synteny blocks (default parameter is 20)
     + dustThreshold controls the upper limit of gene family. The gene family will be filtered when homologous genes exceeding dustThreshold. For above example, target copy numbers are 2,4,2,2,2 in each species, then the dustThreshold is 13 (2+4+2+2+2+1)
3. wait untill the calculations done, copy the results to MAC, and go ahead with next step

A  ***drimm-synteny.bat*** file is like this:
>
> @echo off
>
> REM Loop through each subdirectory
> for /d %%A in ("C:\Users\fengtao\Desktop\drimm_batch\genespace26\*") do (
   >
    > echo Entering directory: %%A\2_DrimmRaw\
    >
    > mkdir "%%A\2_DrimmRaw\"
    >
    > copy "%%A\1_SynOG\drimm.sequence" "%%A\2_DrimmRaw\"
    >
    > pushd "%%A\2_DrimmRaw\"
    >
    >
    > echo inputs like this, modify accordingly: inputfile=drimm.sequence  outdir=.  cycleLengthThreshold=20  dustThreshold=5
    >
    >
    > REM Run your executable tool with non-interactive inputs
    >
    > "C:\Users\fengtao\Desktop\processDrimm\drimm\drimm\bin\Release\netcoreapp3.1\win-x64\drimm.exe"
    >
    > echo %%A done!
    >
    > popd
> )
> pause

In [8]:
# step3 process drimm
# from drimm-synteny, we get raw blocks, and we need to clean and sort the raw blcoks to
## 1. split the all-in-one blocks (drimm-synteny output) into individual genomes
## 2. meanwhile divide the blocks into separate ones based on ratio (normally from genome duplication, but segmental duplications cannot be distinguished, and are also processed)
workdir = "/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/Asteraceae_phylogenomics/pipeline/workdir"
BED_folder = "/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/Asteraceae_phylogenomics/pipeline/"
if workdir[-1] != "/": workdir += "/"
if BED_folder[-1] != "/": BED_folder += "/"

outdir = workdir + "out1/"

sp='sp46'
index_file = "/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/Asteraceae_phylogenomics/pipeline/Smar.EM05.v1.genome.fa.fai"
df_index = make_index(sp,index_file) #your genome index and genome name you want to use
DIC = {f"{row['sp']}@{row['chr']}": row['length'] for _, row in df_index.iterrows()}

genome_meta = make_genome_meta("sp46",1,17)
chromosome_meta = make_chromosome_meta(BED_folder,DIC)

#bed_path = "/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/2_analysis/10_ancestralGenome/bed"
OG_type = "synOG" # synOG|orthofinder
working_dir = outdir
bed_dir = workdir + "bed/"

if working_dir[-1] != "/": working_dir += "/"
if bed_dir[-1] != "/": bed_dir += "/"

target = ["sp46"]
reference = "sp03v5"

for species in target:
    if not species == reference: 
        species_list = [reference, species]
        sp_working_dir = working_dir + species + "/"
        print(sp_working_dir)
        homolog_dir = sp_working_dir + "1_SynOG/"
        if not Path(sp_working_dir).exists():
            print("no homologs found for " + species)
        else:
            block_file = sp_working_dir + "/2_DrimmRaw/blocks.txt"
            sequence = readSequence(block_file)
            drimmSyntenyFile = sp_working_dir + "/2_DrimmRaw/synteny.txt"
            
            list_ = prepare_meta(species_list, genome_meta, chromosome_meta, OG_type, bed_dir)
            sp_list = list_[0]
            sp_ratio = list_[1]
            sp_ratio = ":".join([str(e) for e in sp_ratio]) # convert sp_ratio to a string
            chr_number = list_[2]
            
            drimm_split_blocks_dir,raw_block_dir,result_dir = prepare_dir(sp_working_dir)
            write_raw_blocks(sequence,chr_number,drimm_split_blocks_dir,sp_list)
            processLCSAndFirstFilter = plff.processLCSAndFirstFilter(drimm_split_blocks_dir, raw_block_dir, sp_ratio,
                                                         drimm_split_blocks_dir, homolog_dir, drimmSyntenyFile,
                                                         sp_list, 's')
            processLCSAndFirstFilter.excute()

            processFinalFilter = pff.processFinalFilter(sp_list, raw_block_dir, drimm_split_blocks_dir, result_dir, 's')
            processFinalFilter.excute()
            shutil.rmtree(raw_block_dir)


/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/Asteraceae_phylogenomics/pipeline/workdir/out1/sp46/
288:1:chr_8 has 1 anchors, drop!
318:1:chr_11 has 1 anchors, drop!
512:1:chr_13 has 1 anchors, drop!
85:1:chr_15 has 1 anchors, drop!
72:1:chr_19 has 1 anchors, drop!
636:1:chr_23 has 1 anchors, drop!
109:1:chr_25 has 1 anchors, drop!
69:1:chr_28 has 1 anchors, drop!
52:1:chr_33 has 1 anchors, drop!
52:2:chr_34 has 2 anchors, drop!
56:1:chr_34 has 1 anchors, drop!
56:2:chr_35 has 2 anchors, drop!
66:1:chr_35 has 2 anchors, drop!
723:1:chr_36 has 2 anchors, drop!
48:1:chr_36 has 1 anchors, drop!
48:2:chr_37 has 2 anchors, drop!
770:1:chr_37 has 2 anchors, drop!
770:2:chr_37 has 1 anchors, drop!
226:1:chr_38 has 1 anchors, drop!
430:1:chr_42 has 1 anchors, drop!
257:1:chr_1 has 1 anchors, drop!
298:1:chr_1 has 2 anchors, drop!
297:1:chr_1 has 2 anchors, drop!
664:1:chr_2 has 1 anchors, drop!
553:1:chr_2 has 2 anchors, drop!
498:1:chr

In [3]:
# use grimm2barplot to to do chromosome painting
## use R packages chromoMap to paint blocks on full chromosme background (a difference to iage_plot function);
## 2.1 let's first make a combed file based on normal bed, this special bed will be used by combed_parser() and grimmBlock_parser()
beddir = BED_folder
comBed = make_comBed(beddir)
#comBed.to_csv(workdir + 'combBed.txt', sep='\t', header=True, index=False)

## 2.2 let's read in cleaned dirmm blocks, parse it and generate new data <5_SynBlocks>; no need to worry about ratio as all scenarios will be considered in one run
working_dir = workdir + "out1"
if not working_dir.endswith("/"): working_dir += "/"

target = ["sp46"]

for species in target:
    blockDir = working_dir + species + "/3_DrimmBlocks/finalBlocks/"
    output_path = blockDir + "../../5_SynBlocks/"
    if not os.path.exists(output_path): os.mkdir(output_path)
    dic = combed_parser(comBed)
    blockFiles = [os.path.join(blockDir, filename) for filename in os.listdir(blockDir) if filename.endswith('.synteny.genename')]
    grimmBlock_parser(blockFiles,dic,output_path)

processing sp03v5.bed
processing sp46.bed
processing sp03v5_2 blocks ...
writing sp03v5_2 blocks ...
sp03v5_2 done
processing sp46_2 blocks ...
writing sp46_2 blocks ...
sp46_2 done
processing sp46_1 blocks ...
writing sp46_1 blocks ...
sp46_1 done
processing sp03v5_1 blocks ...
writing sp03v5_1 blocks ...
sp03v5_1 done
processing sp46_3 blocks ...
writing sp46_3 blocks ...
sp46_3 done
processing sp03v5_3 blocks ...
writing sp03v5_3 blocks ...
sp03v5_3 done


In [6]:
## 2.3 take outputs from last step, generate table for plotting in R package <chromoMap>; no need to worry about ratio as all scenarios will be considered in one run

target = ["sp46"]
reference = "sp03v5"
working_dir = workdir + "out1"
if not working_dir.endswith("/"): working_dir += "/"

for species in target:
    blockDir = working_dir + species + "/5_SynBlocks/"
    blocks = [file for file in os.listdir(blockDir) if file.endswith("synBlock.txt")]
    for block in blocks:
        target_block_name = block.split("_")[0]
        target_block_type = block.split("_")[1]
        reference_block = blockDir + reference + "_" + target_block_type + "_synBlock.txt"
        target_block = os.path.join(blockDir, block)
        target_chr = blockDir + "../3_DrimmBlocks/finalBlocks/" + target_block_name + "_" + target_block_type +".final.block"
        
        dic1,dic2 = get_block_info(reference_block,target_block)
        coo_gene,coo_bp = get_chromosome_info(target_chr,dic1,dic2)

        columns = ["chr","haplotype", "start", "end", "ancestral_group"]
        output_path = blockDir + "../6_chromPainting/"
        if not os.path.exists(output_path): os.mkdir(output_path)

        r1 = pd.DataFrame(coo_gene, columns=columns) # make a dataframe from the list of list
        r2 = pd.DataFrame(coo_bp, columns=columns) # make a dataframe from the list of list
        out_put1 = output_path + target_block_name + "_" + target_block_type +"_cooGene.txt"
        out_put2 = output_path + target_block_name + "_" + target_block_type +"_cooBp.txt"
        r1.to_csv(out_put1, sep='\t', header=True, index=False)
        r2.to_csv(out_put2, sep='\t', header=True, index=False)

['670_1_chr_1', '672_1_chr_1', '671_1_chr_1', '669_1_chr_1', '667_1_chr_1', '519_1_chr_1', '523_1_chr_1', '528_1_chr_1', '526_1_chr_1', '765_1_chr_1', '510_1_chr_1', '418_1_chr_1', '422_1_chr_1', '423_1_chr_1', '426_1_chr_1', '433_1_chr_1', '651_1_chr_1', '432_1_chr_1', '431_1_chr_1', '429_1_chr_1', '653_1_chr_1', '411_1_chr_1']
['632_1_chr_2', '603_1_chr_2', '631_1_chr_2', '663_1_chr_2', '662_1_chr_2', '534_1_chr_2', '561_1_chr_2', '564_1_chr_2', '567_1_chr_2', '660_1_chr_2', '794_1_chr_2', '605_1_chr_2', '680_1_chr_2', '600_1_chr_2', '555_1_chr_2', '778_1_chr_2', '486_1_chr_2', '467_1_chr_2', '551_1_chr_2', '541_1_chr_2', '547_1_chr_2', '538_1_chr_2', '543_1_chr_2', '545_1_chr_2', '539_1_chr_2', '549_1_chr_2', '602_1_chr_2', '505_1_chr_2', '494_1_chr_2', '502_1_chr_2', '491_1_chr_2', '487_1_chr_2']
['535_1_chr_3', '533_1_chr_3', '506_1_chr_3', '503_1_chr_3', '532_1_chr_3', '501_1_chr_3', '499_1_chr_3', '781_1_chr_3', '434_1_chr_3', '530_1_chr_3', '425_1_chr_3', '427_1_chr_3', '607_1_

In [6]:
## 2.4 generate inputs for chromoMap
sp='sp46'
index_file = "/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/Asteraceae_phylogenomics/pipeline/Smar.EM05.v1.genome.fa.fai"
df_index = make_index(sp,index_file) #your genome index and genome name you want to use
DIC = {f"{row['sp']}@{row['chr']}": row['length'] for _, row in df_index.iterrows()}

genome_meta = make_genome_meta("sp46",1,17)
chromosome_meta = make_chromosome_meta(BED_folder,DIC)

target = ["sp46"]

workdir = "/Users/fengtao/Library/CloudStorage/OneDrive-WageningenUniversity&Research/project/Asteraceae_evolution/Asteraceae_phylogenomics/pipeline/workdir"
working_dir = workdir + "/out1"
if working_dir[-1] != "/": working_dir += "/"

# obtain information of chromosome number in all species from genome_meta
#genome_meta_df = pd.read_csv(genome_meta, sep='\t', dtype = str, keep_default_na=False, names=['genome', 'ratio1', 'ratio2','chromosome'])
genome_meta_dict = genome_meta.set_index('sp')['chrN'].to_dict()
chromosome_meta_dict = {lst[0]: lst[1:] for lst in chromosome_meta}

for genome in target:
    outdir = working_dir + genome + "/"
    chromosome_list = chromosome_meta_dict[genome]
    chr_info_list = []
    i = 1
    for chromosome in chromosome_list:
        if i <= int(genome_meta_dict[genome]): # make sure only take the long chromosomes as specified in genome_meta
            drimm_chr = "chr_" + chromosome.split("_")[-2]
            genome_chr = "_".join(chromosome.split("_")[0:-2])
            length = chromosome.split("_")[-1]
            chr_info = [drimm_chr,genome_chr,"0",length]
            chr_info_list.append(chr_info)
            i += 1
    df = pd.DataFrame(chr_info_list, columns=['drimm_chr', 'genome_chr', 'start','end'])
    df = df.sort_values(by='genome_chr')
    df.to_csv(outdir+genome+".txt",sep="\t",index=False)